In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark
import pandas as pd
import numpy as np
import pymysql
import seaborn as sns

In [2]:
sc = pyspark.SparkContext('local[*]')

In [3]:
# pip install pandas

In [4]:
# pip install pymysql

In [5]:
# sc.stop()
spark = SparkSession.\
        builder.\
        appName("hello pyspark").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

In [6]:
data = spark.read.csv(path="bank.csv", sep=",", header=True).cache()

In [7]:
data.show()

+---+-------------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+-----------+---+
|age|          job|marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|   poutcome|  y|
+---+-------------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+-----------+---+
| 30|   unemployed|married|  primary|     no|   1787|     no|  no|cellular| 19|  oct|      79|       1|   -1|       0|nonexistent| no|
| 33|     services|married|secondary|     no|   4789|    yes| yes|cellular| 11|  may|     220|       1|  339|       4|    failure| no|
| 35|   management| single| tertiary|     no|   1350|    yes|  no|cellular| 16|  apr|     185|       1|  330|       1|    failure| no|
| 30|   management|married| tertiary|     no|   1476|    yes| yes| unknown|  3|  jun|     199|       4|   -1|       0|nonexistent| no|
| 59|  blue-collar|married|secondary|     no|      0|  

In [8]:
data.printSchema()

root
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [9]:
data.show(5)

+---+-----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+-----------+---+
|age|        job|marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|   poutcome|  y|
+---+-----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+-----------+---+
| 30| unemployed|married|  primary|     no|   1787|     no|  no|cellular| 19|  oct|      79|       1|   -1|       0|nonexistent| no|
| 33|   services|married|secondary|     no|   4789|    yes| yes|cellular| 11|  may|     220|       1|  339|       4|    failure| no|
| 35| management| single| tertiary|     no|   1350|    yes|  no|cellular| 16|  apr|     185|       1|  330|       1|    failure| no|
| 30| management|married| tertiary|     no|   1476|    yes| yes| unknown|  3|  jun|     199|       4|   -1|       0|nonexistent| no|
| 59|blue-collar|married|secondary|     no|      0|    yes|  no| unkn

In [10]:
data.head(5)

[Row(age='30', job='unemployed', marital='married', education='primary', default='no', balance='1787', housing='no', loan='no', contact='cellular', day='19', month='oct', duration='79', campaign='1', pdays='-1', previous='0', poutcome='nonexistent', y='no'),
 Row(age='33', job='services', marital='married', education='secondary', default='no', balance='4789', housing='yes', loan='yes', contact='cellular', day='11', month='may', duration='220', campaign='1', pdays='339', previous='4', poutcome='failure', y='no'),
 Row(age='35', job='management', marital='single', education='tertiary', default='no', balance='1350', housing='yes', loan='no', contact='cellular', day='16', month='apr', duration='185', campaign='1', pdays='330', previous='1', poutcome='failure', y='no'),
 Row(age='30', job='management', marital='married', education='tertiary', default='no', balance='1476', housing='yes', loan='yes', contact='unknown', day='3', month='jun', duration='199', campaign='4', pdays='-1', previous='

In [11]:
data.count()

4521

In [12]:
data.describe(["age"]).show()

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|              4521|
|   mean| 41.17009511170095|
| stddev|10.576210958711263|
|    min|                19|
|    max|                87|
+-------+------------------+



In [13]:
for col in data.columns:
    print(col,data.filter(data[col].isNull()).count())

age 0
job 0
marital 0
education 0
default 0
balance 0
housing 0
loan 0
contact 0
day 0
month 0
duration 0
campaign 0
pdays 0
previous 0
poutcome 0
y 0


In [14]:
data=data[data.education != 'unknown']

In [15]:
data = data[data.contact != 'unknown']

In [16]:
data.count()

3083

In [17]:
data.createOrReplaceTempView("Bank")

In [18]:
spark.sql("SELECT * FROM Bank WHERE  poutcome == 'other'").count()

184

In [19]:
data = spark.sql("SELECT * FROM Bank WHERE NOT  poutcome == 'other'")

In [20]:
spark.sql("SELECT * FROM Bank WHERE  pdays < 0").count()

2315

In [21]:
data = data.drop("pdays")

In [24]:
data.show()

+---+-------------+--------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+--------+-----------+---+
|age|          job| marital|education|default|balance|housing|loan|  contact|day|month|duration|campaign|previous|   poutcome|  y|
+---+-------------+--------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+--------+-----------+---+
| 30|   unemployed| married|  primary|     no|   1787|     no|  no| cellular| 19|  oct|      79|       1|       0|nonexistent| no|
| 33|     services| married|secondary|     no|   4789|    yes| yes| cellular| 11|  may|     220|       1|       4|    failure| no|
| 35|   management|  single| tertiary|     no|   1350|    yes|  no| cellular| 16|  apr|     185|       1|       1|    failure| no|
| 35|   management|  single| tertiary|     no|    747|     no|  no| cellular| 23|  feb|     141|       2|       3|    failure| no|
| 39|   technician| married|secondary|     no|    147|    yes|  no| cellular|  6|  

In [28]:
data.createOrReplaceTempView("Bank")

In [30]:
df = spark.sql("SELECT * FROM Bank").toPandas()

In [31]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,0,nonexistent,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,1,failure,no
3,35,management,single,tertiary,no,747,no,no,cellular,23,feb,141,2,3,failure,no
4,39,technician,married,secondary,no,147,yes,no,cellular,6,may,151,2,0,nonexistent,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2894,49,blue-collar,married,secondary,no,322,no,no,cellular,14,aug,356,2,0,nonexistent,no
2895,38,blue-collar,married,secondary,no,1205,yes,no,cellular,20,apr,45,4,1,failure,no
2896,32,services,single,secondary,no,473,yes,no,cellular,7,jul,624,5,0,nonexistent,no
2897,33,services,married,secondary,no,-333,yes,no,cellular,30,jul,329,5,0,nonexistent,no


In [32]:
df['deposit_yes'] = np.where(df['y']=='yes', '1', '0')

In [33]:
df['deposit_no'] = np.where(df['y']=='no', '1', '0')

In [34]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,previous,poutcome,y,deposit_yes,deposit_no
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,0,nonexistent,no,0,1
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,4,failure,no,0,1
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,1,failure,no,0,1
3,35,management,single,tertiary,no,747,no,no,cellular,23,feb,141,2,3,failure,no,0,1
4,39,technician,married,secondary,no,147,yes,no,cellular,6,may,151,2,0,nonexistent,no,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2894,49,blue-collar,married,secondary,no,322,no,no,cellular,14,aug,356,2,0,nonexistent,no,0,1
2895,38,blue-collar,married,secondary,no,1205,yes,no,cellular,20,apr,45,4,1,failure,no,0,1
2896,32,services,single,secondary,no,473,yes,no,cellular,7,jul,624,5,0,nonexistent,no,0,1
2897,33,services,married,secondary,no,-333,yes,no,cellular,30,jul,329,5,0,nonexistent,no,0,1


In [35]:
df = df.drop(columns=['y'])

In [36]:
df['month'] = pd.Categorical(df['month'] ,['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'])

In [37]:
data1 = df.sort_values("month")

In [38]:
data1.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,previous,poutcome,deposit_yes,deposit_no
2583,71,unknown,married,tertiary,no,353,no,no,cellular,27,jan,208,1,2,failure,0,1
1981,30,services,single,secondary,no,334,no,no,cellular,29,jan,389,2,0,nonexistent,0,1
1232,29,services,single,secondary,no,192,no,no,cellular,30,jan,327,4,0,nonexistent,0,1
1234,49,management,married,tertiary,no,87,no,no,cellular,29,jan,165,1,0,nonexistent,0,1
2547,66,retired,married,tertiary,no,1948,no,no,cellular,28,jan,216,1,4,success,1,0


In [39]:
from sqlalchemy import create_engine

In [40]:
# pip install sqlalchemy

In [41]:
username = "root"
password = "dstb"
port = 3306
database = "dstb_db"

In [42]:
pymysql.install_as_MySQLdb()

In [43]:
engine = create_engine('mysql+mysqldb://%s:%s@db:%i/%s'%(username, password, port, database))

In [44]:
data1.to_sql('Bank', engine, if_exists='replace')

In [45]:
#  pip install ipython-sql

In [46]:
%load_ext sql

In [47]:
%reload_ext sql

In [48]:
%sql mysql://root:dstb@db:3306/dstb_db

In [49]:
result = %sql select * from Bank

 * mysql://root:***@db:3306/dstb_db
2899 rows affected.


In [50]:
%sql select * from Bank limit 10

 * mysql://root:***@db:3306/dstb_db
10 rows affected.


index,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,previous,poutcome,deposit_yes,deposit_no
2583,71,unknown,married,tertiary,no,353,no,no,cellular,27,jan,208,1,2,failure,0,1
1981,30,services,single,secondary,no,334,no,no,cellular,29,jan,389,2,0,nonexistent,0,1
1232,29,services,single,secondary,no,192,no,no,cellular,30,jan,327,4,0,nonexistent,0,1
1234,49,management,married,tertiary,no,87,no,no,cellular,29,jan,165,1,0,nonexistent,0,1
2547,66,retired,married,tertiary,no,1948,no,no,cellular,28,jan,216,1,4,success,1,0
2735,45,blue-collar,married,primary,no,185,yes,no,cellular,29,jan,249,3,0,nonexistent,0,1
1236,40,management,single,tertiary,no,1361,no,no,cellular,29,jan,1473,1,0,nonexistent,1,0
1530,48,blue-collar,married,primary,no,1678,no,no,cellular,28,jan,263,2,0,nonexistent,0,1
884,40,technician,single,tertiary,no,226,no,no,cellular,15,jan,176,2,0,nonexistent,0,1
892,43,management,divorced,tertiary,no,99,no,no,cellular,29,jan,24,3,0,nonexistent,0,1


In [51]:
result = %sql select * from Bank 

 * mysql://root:***@db:3306/dstb_db
2899 rows affected.


In [52]:
result

index,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,previous,poutcome,deposit_yes,deposit_no
2583,71,unknown,married,tertiary,no,353,no,no,cellular,27,jan,208,1,2,failure,0,1
1981,30,services,single,secondary,no,334,no,no,cellular,29,jan,389,2,0,nonexistent,0,1
1232,29,services,single,secondary,no,192,no,no,cellular,30,jan,327,4,0,nonexistent,0,1
1234,49,management,married,tertiary,no,87,no,no,cellular,29,jan,165,1,0,nonexistent,0,1
2547,66,retired,married,tertiary,no,1948,no,no,cellular,28,jan,216,1,4,success,1,0
2735,45,blue-collar,married,primary,no,185,yes,no,cellular,29,jan,249,3,0,nonexistent,0,1
1236,40,management,single,tertiary,no,1361,no,no,cellular,29,jan,1473,1,0,nonexistent,1,0
1530,48,blue-collar,married,primary,no,1678,no,no,cellular,28,jan,263,2,0,nonexistent,0,1
884,40,technician,single,tertiary,no,226,no,no,cellular,15,jan,176,2,0,nonexistent,0,1
892,43,management,divorced,tertiary,no,99,no,no,cellular,29,jan,24,3,0,nonexistent,0,1


In [53]:
type(result)

sql.run.ResultSet

In [54]:
result = result.DataFrame()

In [55]:
result.head()

,index,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,previous,poutcome,deposit_yes,deposit_no
0,2583,71,unknown,married,tertiary,no,353,no,no,cellular,27,jan,208,1,2,failure,0,1
1,1981,30,services,single,secondary,no,334,no,no,cellular,29,jan,389,2,0,nonexistent,0,1
2,1232,29,services,single,secondary,no,192,no,no,cellular,30,jan,327,4,0,nonexistent,0,1
3,1234,49,management,married,tertiary,no,87,no,no,cellular,29,jan,165,1,0,nonexistent,0,1
4,2547,66,retired,married,tertiary,no,1948,no,no,cellular,28,jan,216,1,4,success,1,0
